In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai

In [4]:
issue= "Shower won't turn off water leaking"
cx_prompt= "Act as a customer whose facing a plumbing problem. Answer the questions asked by the plumber.\n\nThe issue you are facing is:"+ issue
plumber_prompt= "You are a plumber in the UK. You need to ask short questions to the customer and give a price quote.\nPlease proceed step by step. Ask only one question at a time, and then ask the next one."
p=[issue]
messages= [{"role":"system", "content": plumber_prompt}]
messages.append({"role":"user","content": issue})
chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", messages=messages
        )
reply = chat.choices[0].message.content
messages.append({"role": "assistant", "content": reply})
transcript=[chat]
tokens= [chat.usage.total_tokens]
#print(messages)
p.append(reply)
h=[reply]
cx_messages= [{"role":"system", "content": cx_prompt}]
i=0
while i<=15:
    if "£" not in h[i] and i<20:
        cx_messages.append({"role":"user", "content": h[i]})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", messages= cx_messages
        )
        res = response.choices[0].message.content
        cx_messages.append({"role": "assistant", "content": res})
        p.append(res)
        transcript.append(response)
        tokens.append(response.usage.total_tokens)
        
        messages.append({"role":"user","content": res})
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", messages= messages
        )
        replies = result.choices[0].message.content
        messages.append({"role":"assistant","content": replies})
        
        transcript.append(result)
        tokens.append(result.usage.total_tokens)
        h.append(replies)
        p.append(replies)
        #print(cx_messages)
        #print("*"*100)

    else:
        break

    i+=1



In [7]:
import pandas as pd
df= pd.DataFrame(p,columns=["messages"])
transcripts= [0]
for i in transcript:
    transcripts.append(i)
tok=[0]
for i in tokens:
    tok.append(i)
df["output"] = transcripts
df["total_tokens"] = tok
df.head()

,messages,output,total_tokens
0,Shower won't turn off water leaking,0,0
1,Is the leak coming from the showerhead itself ...,{'id': 'chatcmpl-7T7FxvwPXYrE14cePwq6sAICphehd...,81
2,I believe the water is leaking from a differen...,{'id': 'chatcmpl-7T7FziskfV8bFmj7A5G9sCnlalxsR...,81
3,Is the water leaking from the shower faucet or...,{'id': 'chatcmpl-7T7G1U0FrO1fUqXRYgBpUMDoAJWSk...,120
4,I think the water is leaking from the shower f...,{'id': 'chatcmpl-7T7G3zpTPgR66jOgoPACtxMqXTKU3...,117


In [ ]:
df.to_csv("outputz1.csv")